# Methods to Detect Center

## Method 1: Masking Low NDVI + Center of Mass

In [4]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        if true_color.max() > 0:
            true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color


# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 1 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode1 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            # Menyimpan data untuk menghubungkan koordinat piksel dengan koordinat dunia nyata
            # Yang disimpan:
            # Resolutsi citra: brp besar 1 piksel mewakili di dunia nyata dalam sumbu x dan y
            # Orientasi citra: diputar/miring
            # Posisi citra dalam sistem koordinat dunia nyata 
            transform = src.transform

        # # Perhitungan NDVI
        # NDVI (Normalized Difference Vegetation Index) adalah indeks yang digunakan untuk mendeteksi keberadaan vegetasi dengan membandingkan band NIR (Near Infrared) dan Red.
        # Rumus NDVI adalah: NDVI = (NIR - Red) / (NIR + Red)
        # Nilai NDVI berkisar antara -1 hingga 1, di mana nilai tinggi menunjukkan vegetasi yang lebat, dan nilai rendah (terutama mendekati -1) menunjukkan area tanpa vegetasi, seperti tanah atau air.
        # Fungsi ini digunakan untuk mendeteksi area vegetasi dan area yang mungkin merupakan pusat gunung berapi, di mana vegetasi lebih sedikit.

        with np.errstate(divide='ignore', invalid='ignore'):
            # Formula NDVI = (NIR - Red) / (NIR + Red)
            ndvi = (B8 - B4) / (B8 + B4)
            # Menghindari pembagian dengan 0 di tempat di mana (NIR + Red) sama dengan 0
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0

        # # Clipping nilai NDVI agar berada dalam rentang yang valid
        # Nilai NDVI yang lebih kecil dari -1 atau lebih besar dari 1 akan dipotong (clipped) ke dalam rentang tersebut.
        # Ini untuk menghindari data yang tidak sesuai atau anomali yang bisa terjadi selama perhitungan NDVI.
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])  # Menyaring nilai NaN
        valid_ndvi = flat_ndvi[mask_valid]

        # # KMeans Clustering
        # KMeans digunakan untuk mengelompokkan piksel berdasarkan nilai NDVI. Dengan menggunakan 2 cluster, kita ingin membedakan area dengan NDVI rendah (misalnya area tanpa vegetasi atau daerah kawah gunung berapi) dan NDVI tinggi (vegetasi).
        # Pengelompokan ini penting untuk menandai area dengan nilai NDVI rendah sebagai kandidat untuk pusat gunung berapi.
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah, yang biasanya mewakili area sekitar kawah atau puncak gunung berapi.
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)  # Cluster dengan nilai NDVI terendah
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster  # Masker untuk area NDVI rendah

        # # Menghitung Center of Mass
        # Center of mass adalah titik rata-rata dari distribusi massa pada objek dalam citra. Pada konteks ini, center of mass digunakan untuk menentukan titik pusat dari area NDVI rendah.
        # Setelah kita memiliki mask dengan cluster NDVI rendah, kita dapat menghitung titik pusat dari area tersebut menggunakan fungsi center_of_mass dari scipy.ndimage.
        # Ini dilakukan untuk mengetahui lokasi yang tepat dari pusat potensi gunung berapi berdasarkan citra NDVI.
        cy, cx = center_of_mass(ndvi_mask)
        cy, cx = int(cy), int(cx)  # Mengonversi koordinat ke integer

        # Mengonversi koordinat piksel (cy, cx) menjadi koordinat geospasial (latitude, longitude) berdasarkan transformasi citra
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi (latitude dan longitude) ke dalam dictionary untuk digunakan lebih lanjut
        hasil_deteksi_metode1[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Clustering KMeans
        axs[0, 1].imshow(labels.reshape(ndvi.shape), cmap='coolwarm')
        axs[0, 1].set_title('Hasil Clustering KMeans', fontsize=14)
        axs[0, 1].axis('off')

        # Mask Cluster NDVI Rendah
        axs[1, 0].imshow(ndvi_mask, cmap='gray')
        axs[1, 0].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # True Color
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')


        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()


# Menyimpan hasil deteksi ke dalam file CSV
import csv

csv_output_path = os.path.join(output_dir, 'hasil_deteksi_metode1.csv')

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['nama_gunung', 'koordinat_hasil_deteksi (lat, lon)'])
    for nama, data in hasil_deteksi_metode1.items():
        koordinat = f"{data['detected_lat']}, {data['detected_lon']}"
        writer.writerow([nama, koordinat])

## Method 2: Masking Low NDVI + Largest Blob + Center of Mass

In [5]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
from skimage.measure import label, regionprops

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        if true_color.max() > 0:
            true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color

# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 2 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode2 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            transform = src.transform

        # Perhitungan NDVI
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (B8 - B4) / (B8 + B4)
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])
        valid_ndvi = flat_ndvi[mask_valid]

        # KMeans Clustering
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

        # Deteksi Largest Blob (komponen terbesar)
        labeled_mask = label(ndvi_mask)
        regions = regionprops(labeled_mask)
        largest_region = max(regions, key=lambda r: r.area)
        largest_blob_mask = labeled_mask == largest_region.label

        # Menghitung Center of Mass dari Blob Terbesar
        cy, cx = center_of_mass(largest_blob_mask)
        cy, cx = int(cy), int(cx)

        # Mengonversi koordinat piksel menjadi koordinat geospasial
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi
        hasil_deteksi_metode2[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        # Membuat visualisasi hasil deteksi
        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Mask Cluster NDVI Rendah
        axs[0, 1].imshow(ndvi_mask, cmap='gray')
        axs[0, 1].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[0, 1].axis('off')

        # Menambahkan legenda manual untuk mask NDVI rendah
        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[0, 1].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # Largest Blob
        axs[1, 0].imshow(largest_blob_mask, cmap='gray')
        axs[1, 0].set_title('Largest Blob (Area Terbesar)', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='Blob Terbesar'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)


        # True Color + Titik Pusat
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')

        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()

# Menyimpan hasil deteksi ke dalam file CSV
import csv

csv_output_path = os.path.join(output_dir, 'hasil_deteksi_metode2.csv')

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['nama_gunung', 'koordinat_deteksi (lat, lon)'])
    for nama, data in hasil_deteksi_metode1.items():
        koordinat = f"({data['detected_lat']}, {data['detected_lon']})"
        writer.writerow([nama, koordinat])


## Method 3: Masking Low NDVI + Blob Closest to Center + Center of Mass

In [6]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
from skimage.measure import label, regionprops

# Fungsi untuk membuat citra true color dari citra Sentinel
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        # Membaca band-band citra
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue

        # Membuat citra true color dengan menggabungkan tiga band
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)  # Mengganti NaN dengan 0
        true_color = true_color / true_color.max()  # Menormalisasi citra
        return true_color

# Direktori input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir = 'Metode 3 (TA)'
os.makedirs(output_dir, exist_ok=True)

# Untuk menyimpan hasil deteksi pusat tiap gunung
hasil_deteksi_metode3 = {}

# Loop untuk setiap file .tif
for filename in os.listdir(input_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        filepath = os.path.join(input_dir, filename)
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')

        with rasterio.open(filepath) as src:
            # Membaca band-band yang dibutuhkan (Red dan NIR)
            B4 = src.read(4).astype('float32')  # Red
            B8 = src.read(8).astype('float32')  # NIR
            transform = src.transform

        # Perhitungan NDVI
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (B8 - B4) / (B8 + B4)
            ndvi[(B8 + B4) == 0] = np.nan  # Hindari pembagian dengan 0
        ndvi = np.clip(ndvi, -1, 1)

        # Meratakan citra NDVI menjadi array 1D dan membuat masker valid untuk nilai yang bukan NaN
        flat_ndvi = ndvi.reshape(-1, 1)
        mask_valid = ~np.isnan(flat_ndvi[:, 0])
        valid_ndvi = flat_ndvi[mask_valid]

        # KMeans Clustering
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(valid_ndvi)

        # Menentukan cluster dengan nilai NDVI rendah
        cluster_means = [valid_ndvi[labels[mask_valid] == i].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

        # Deteksi semua Blob (komponen terhubung)
        labeled_mask = label(ndvi_mask)
        regions = regionprops(labeled_mask)

        # Menentukan titik tengah gambar
        center_y, center_x = np.array(ndvi.shape) / 2

        # Cari blob yang paling dekat ke titik tengah gambar
        min_distance = np.inf
        closest_region = None
        for region in regions:
            # Menghitung center of mass masing-masing blob
            blob_cy, blob_cx = region.centroid
            distance = np.sqrt((blob_cy - center_y) ** 2 + (blob_cx - center_x) ** 2)
            if distance < min_distance:
                min_distance = distance
                closest_region = region

        # Membuat mask untuk blob terdekat
        closest_blob_mask = labeled_mask == closest_region.label

        # Menghitung Center of Mass dari Blob Terdekat
        cy, cx = center_of_mass(closest_blob_mask)
        cy, cx = int(cy), int(cx)

        # Mengonversi koordinat piksel menjadi koordinat geospasial
        lon, lat = rasterio.transform.xy(transform, cy, cx)

        # Menyimpan hasil deteksi
        hasil_deteksi_metode3[gunung_name] = {
            'detected_lat': lat,
            'detected_lon': lon
        }

        # Buat citra true color untuk visualisasi
        true_color = create_true_color_image(filepath)

        # Membuat visualisasi hasil deteksi
        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)

        # NDVI Map
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map', fontsize=14)
        axs[0, 0].axis('off')

        # Mask Cluster NDVI Rendah
        axs[0, 1].imshow(ndvi_mask, cmap='gray')
        axs[0, 1].set_title('Mask Cluster NDVI Rendah', fontsize=14)
        axs[0, 1].axis('off')

        # Menambahkan legenda manual untuk mask NDVI rendah
        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='NDVI Rendah (Area Pusat)'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[0, 1].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # Blob Closest to Center
        axs[1, 0].imshow(closest_blob_mask, cmap='gray')
        axs[1, 0].set_title('Blob Closest to Center', fontsize=14)
        axs[1, 0].axis('off')

        legend_elements = [
            Patch(facecolor='white', edgecolor='black', label='Blob Terdekat ke Pusat'),
            Patch(facecolor='black', edgecolor='black', label='Area Lainnya')
        ]
        axs[1, 0].legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=10)

        # True Color + Titik Pusat
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Pusat Gunung')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat', fontsize=14)
        axs[1, 1].axis('off')

        # Judul besar
        fig.suptitle(f'Deteksi Pusat Gunung {gunung_name}\nKoordinat: {lat}, {lon}', fontsize=16, fontweight='bold')

        # Save
        output_path = os.path.join(output_dir, f'{gunung_name}.png')
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()

# Menyimpan hasil deteksi ke dalam file CSV
import csv

csv_output_path = os.path.join(output_dir, 'hasil_deteksi_metode3.csv')

with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['nama_gunung', 'koordinat_deteksi (lat, lon)'])
    for nama, data in hasil_deteksi_metode3.items():
        koordinat = f"({data['detected_lat']}, {data['detected_lon']})"
        writer.writerow([nama, koordinat])


## Method 4: Integrasi DEM sebagai Fitur Kedua untuk Method 1 

In [13]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
import csv

# Fungsi membuat citra True Color
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)
        if true_color.max() > 0:
            true_color /= true_color.max()
        return true_color

# Folder input dan output
sentinel_dir = '../DATA/SENTINEL 2/'
dem_dir = '../DATA/DEM/'
output_dir = 'Metode 4'
os.makedirs(output_dir, exist_ok=True)

hasil_deteksi_metode4 = {}

# Loop semua file Sentinel2
for filename in os.listdir(sentinel_dir):
    if filename.endswith('.tif') and filename.startswith('Sentinel2_'):
        gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')
        sentinel_path = os.path.join(sentinel_dir, filename)
        dem_path = os.path.join(dem_dir, f'DEM_{gunung_name}.tif')

        if not os.path.exists(dem_path):
            print(f"DEM tidak ditemukan untuk {gunung_name}, lewati...")
            continue

        # Baca band Sentinel
        with rasterio.open(sentinel_path) as src:
            B4 = src.read(4).astype('float32')
            B8 = src.read(8).astype('float32')
            transform = src.transform
            crs = src.crs
            width, height = src.width, src.height

        # Hitung NDVI
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (B8 - B4) / (B8 + B4)
            ndvi[(B8 + B4) == 0] = np.nan
        ndvi = np.clip(ndvi, -1, 1)

        # Baca DEM & resample jika ukuran beda
        with rasterio.open(dem_path) as dem_src:
            dem_data = dem_src.read(1).astype('float32')
            if dem_data.shape != ndvi.shape:
                from rasterio.warp import reproject, Resampling
                dem_resampled = np.empty_like(ndvi)
                reproject(
                    source=dem_data,
                    destination=dem_resampled,
                    src_transform=dem_src.transform,
                    src_crs=dem_src.crs,
                    dst_transform=transform,
                    dst_crs=crs,
                    resampling=Resampling.bilinear
                )
            else:
                dem_resampled = dem_data

        # Gabung NDVI dan DEM untuk clustering
        flat_ndvi = ndvi.reshape(-1)
        flat_dem = dem_resampled.reshape(-1)
        mask_valid = ~np.isnan(flat_ndvi) & ~np.isnan(flat_dem)
        features = np.stack([flat_ndvi[mask_valid], flat_dem[mask_valid]], axis=1)

        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        labels = np.zeros_like(flat_ndvi, dtype=np.uint8)
        labels[mask_valid] = kmeans.fit_predict(features)

        # Pilih cluster dengan NDVI terendah
        cluster_means = [features[labels[mask_valid] == i][:, 0].mean() for i in range(2)]
        low_ndvi_cluster = np.argmin(cluster_means)
        ndvi_mask = labels.reshape(ndvi.shape) == low_ndvi_cluster

        # Titik pusat (center of mass)
        cy, cx = center_of_mass(ndvi_mask)
        cy, cx = int(cy), int(cx)
        lon, lat = rasterio.transform.xy(transform, cy, cx)
        hasil_deteksi_metode4[gunung_name] = {'detected_lat': lat, 'detected_lon': lon}

        # Visualisasi 2×2
        true_color = create_true_color_image(sentinel_path)
        fig, axs = plt.subplots(2, 2, figsize=(10, 10), constrained_layout=True)
        axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
        axs[0, 0].set_title('NDVI Map'); axs[0, 0].axis('off')
        axs[0, 1].imshow(labels.reshape(ndvi.shape), cmap='coolwarm')
        axs[0, 1].set_title('KMeans Clustering'); axs[0, 1].axis('off')
        axs[1, 0].imshow(ndvi_mask, cmap='gray')
        axs[1, 0].set_title('NDVI Rendah Mask'); axs[1, 0].axis('off')
        axs[1, 1].imshow(true_color)
        axs[1, 1].scatter(cx, cy, c='red', s=50, label='Titik Pusat')
        axs[1, 1].legend()
        axs[1, 1].set_title('True Color + Titik Pusat')
        axs[1, 1].axis('off')
        fig.suptitle(f'{gunung_name} | Metode 4\nKoordinat: {lat:.5f}, {lon:.5f}', fontsize=16)
        plt.savefig(os.path.join(output_dir, f'{gunung_name}.png'))
        plt.close()

# Simpan hasil ke CSV
csv_path = os.path.join(output_dir, 'hasil_deteksi_metode4.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['nama_gunung', 'lat', 'lon'])
    for name, coord in hasil_deteksi_metode4.items():
        writer.writerow([name, coord['detected_lat'], coord['detected_lon']])


## Method 5: Integrasi Ketinggian untuk Double Clustering pada Method 1

In [14]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
from matplotlib.patches import Patch
from skimage.measure import label  # (opsional, tidak dipakai sekarang)
import csv

# -----------------------------
# Util: True color dari Sentinel-2
# -----------------------------
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)
        mx = true_color.max()
        if mx > 0:
            true_color = true_color / mx
        return true_color

# -----------------------------
# Path I/O
# -----------------------------
sentinel_dir = '../DATA/SENTINEL 2/'
dem_dir = '../DATA/DEM/'
output_dir = 'Metode 5'
os.makedirs(output_dir, exist_ok=True)

hasil_deteksi_metode5 = {}

# -----------------------------
# Loop semua file Sentinel-2
# -----------------------------
for filename in os.listdir(sentinel_dir):
    if not (filename.endswith('.tif') and filename.startswith('Sentinel2_')):
        continue

    gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')
    sentinel_path = os.path.join(sentinel_dir, filename)
    dem_path = os.path.join(dem_dir, f'DEM_{gunung_name}.tif')

    if not os.path.exists(dem_path):
        print(f"[SKIP] DEM tidak ditemukan untuk {gunung_name}")
        continue

    # ---------- Baca Sentinel (NDVI) ----------
    with rasterio.open(sentinel_path) as src:
        B4 = src.read(4).astype('float32')  # Red
        B8 = src.read(8).astype('float32')  # NIR
        transform = src.transform
        crs = src.crs

    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (B8 - B4) / (B8 + B4)
        ndvi[(B8 + B4) == 0] = np.nan
    ndvi = np.clip(ndvi, -1, 1)

    # ---------- Baca DEM & samakan grid ----------
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1).astype('float32')

        if dem_data.shape != ndvi.shape or dem_src.transform != transform or dem_src.crs != crs:
            from rasterio.warp import reproject, Resampling
            dem_resampled = np.full_like(ndvi, np.nan, dtype='float32')
            reproject(
                source=dem_data,
                destination=dem_resampled,
                src_transform=dem_src.transform,
                src_crs=dem_src.crs,
                dst_transform=transform,
                dst_crs=crs,
                resampling=Resampling.bilinear
            )
        else:
            dem_resampled = dem_data

    # ==========================
    # 1) K-MEANS NDVI (2 cluster)
    # ==========================
    flat_ndvi = ndvi.reshape(-1, 1)
    valid_ndvi_mask = ~np.isnan(flat_ndvi[:, 0])
    valid_ndvi_vals = flat_ndvi[valid_ndvi_mask]

    if valid_ndvi_vals.size == 0:
        print(f"[SKIP] Semua NDVI NaN: {gunung_name}")
        continue

    kmeans_ndvi = KMeans(n_clusters=2, random_state=42, n_init=10)
    ndvi_labels_flat = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
    ndvi_labels_flat[valid_ndvi_mask] = kmeans_ndvi.fit_predict(valid_ndvi_vals)

    # pilih cluster dengan mean NDVI terendah
    ndvi_means = []
    for i in range(2):
        vals = valid_ndvi_vals[ndvi_labels_flat[valid_ndvi_mask] == i]
        ndvi_means.append(vals.mean() if vals.size > 0 else np.inf)
    low_ndvi_cluster = int(np.argmin(ndvi_means))

    ndvi_low_mask = (ndvi_labels_flat.reshape(ndvi.shape) == low_ndvi_cluster)

    # ==========================
    # 2) K-MEANS DEM dalam area NDVI rendah
    # ==========================
    dem_on_low_ndvi = np.where(ndvi_low_mask, dem_resampled, np.nan)
    flat_dem = dem_on_low_ndvi.reshape(-1, 1)
    valid_dem_mask = ~np.isnan(flat_dem[:, 0])
    valid_dem_vals = flat_dem[valid_dem_mask]

    if valid_dem_vals.size == 0:
        print(f"[SKIP] Tidak ada piksel valid untuk DEM di area NDVI rendah: {gunung_name}")
        continue

    kmeans_dem = KMeans(n_clusters=2, random_state=42, n_init=10)
    dem_labels_flat = np.zeros_like(flat_dem[:, 0], dtype=np.uint8)
    dem_labels_flat[valid_dem_mask] = kmeans_dem.fit_predict(valid_dem_vals)

    # pilih cluster DEM dengan rata-rata ketinggian lebih tinggi
    dem_means = []
    for i in range(2):
        vals = valid_dem_vals[dem_labels_flat[valid_dem_mask] == i]
        dem_means.append(vals.mean() if vals.size > 0 else -np.inf)
    high_dem_cluster = int(np.argmax(dem_means))

    # final mask = NDVI rendah ∩ DEM tinggi
    dem_high_mask = (dem_labels_flat.reshape(ndvi.shape) == high_dem_cluster)
    final_mask = ndvi_low_mask & dem_high_mask

    if not np.any(final_mask):
        print(f"[WARN] Final mask kosong: {gunung_name}. Pusat tidak dapat ditentukan.")
        continue

    # ==========================
    # 3) CENTER OF MASS dari semua piksel final
    # ==========================
    cy, cx = center_of_mass(final_mask.astype(np.uint8))
    if np.isnan(cy) or np.isnan(cx):
        print(f"[WARN] Center of mass tidak valid: {gunung_name}")
        continue
    cy_i, cx_i = int(round(cy)), int(round(cx))
    lon, lat = rasterio.transform.xy(transform, cy_i, cx_i)

    hasil_deteksi_metode5[gunung_name] = {'detected_lat': float(lat), 'detected_lon': float(lon)}

    # ==========================
    # 4) VISUALISASI 2×3
    # ==========================
    true_color = create_true_color_image(sentinel_path)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10), constrained_layout=True)

    # (1) NDVI map
    axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    axs[0, 0].set_title('NDVI Map'); axs[0, 0].axis('off')

    # (2) Mask NDVI rendah (hasil clustering)
    axs[0, 1].imshow(ndvi_low_mask, cmap='gray')
    axs[0, 1].set_title('Mask NDVI Rendah (KMeans)'); axs[0, 1].axis('off')

    # (3) Peta ketinggian (DEM)
    axs[0, 2].imshow(dem_resampled, cmap='terrain')
    axs[0, 2].set_title('Peta Ketinggian (DEM)'); axs[0, 2].axis('off')

    # (4) KMeans DEM (label) — hanya area NDVI rendah
    dem_labels_img = np.full(ndvi.shape, np.nan, dtype='float32')
    dem_labels_img.flat[:] = np.nan
    dem_labels_img = dem_labels_img.reshape(ndvi.shape)
    dem_labels_img_flat = dem_labels_img.reshape(-1)
    dem_labels_img_flat[valid_dem_mask] = dem_labels_flat[valid_dem_mask]
    dem_labels_img = dem_labels_img_flat.reshape(ndvi.shape)
    axs[1, 0].imshow(dem_labels_img, cmap='viridis')
    axs[1, 0].set_title('KMeans DEM (dalam NDVI Rendah)'); axs[1, 0].axis('off')

    # (5) Mask DEM tinggi (final mask)
    axs[1, 1].imshow(final_mask, cmap='gray')
    axs[1, 1].set_title('Mask DEM Tinggi (Final)'); axs[1, 1].axis('off')

    # (6) True color + titik pusat
    axs[1, 2].imshow(true_color)
    axs[1, 2].scatter(cx, cy, c='red', s=50, label='Titik Pusat')
    axs[1, 2].legend()
    axs[1, 2].set_title('True Color + Titik Pusat'); axs[1, 2].axis('off')

    fig.suptitle(f'{gunung_name} | Metode 5\nKoordinat: {lat:.5f}, {lon:.5f}', fontsize=16)
    plt.savefig(os.path.join(output_dir, f'{gunung_name}.png'), dpi=150, bbox_inches='tight')
    plt.close()

# -----------------------------
# Simpan CSV
# -----------------------------
csv_path = os.path.join(output_dir, 'hasil_deteksi_metode5.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['nama_gunung', 'lat', 'lon'])
    for name, coord in hasil_deteksi_metode5.items():
        writer.writerow([name, coord['detected_lat'], coord['detected_lon']])


## Method 6: Combined mask NDVI + Thermal + DEM Cluster

In [3]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage import center_of_mass
import csv

# -----------------------------
# Util: True color dari Sentinel-2
# -----------------------------
def create_true_color_image(image_path):
    with rasterio.open(image_path) as src:
        band4 = src.read(4).astype('float32')  # Red
        band3 = src.read(3).astype('float32')  # Green
        band2 = src.read(2).astype('float32')  # Blue
        true_color = np.dstack((band4, band3, band2))
        true_color = np.nan_to_num(true_color, nan=0)
        mx = true_color.max()
        if mx > 0:
            true_color = true_color / mx
        return true_color

# -----------------------------
# Path I/O
# -----------------------------
sentinel_dir = '../DATA/SENTINEL 2/'
dem_dir = '../DATA/DEM/'
thermal_dir = '../DATA/LANDSAT_30_CF_2Y/'
output_dir = 'Metode 6_NEW VIZ'
os.makedirs(output_dir, exist_ok=True)

hasil_deteksi_metode6 = {}

# -----------------------------
# Loop semua file Sentinel-2
# -----------------------------
for filename in os.listdir(sentinel_dir):
    if not (filename.endswith('.tif') and filename.startswith('Sentinel2_')):
        continue

    gunung_name = filename.replace('Sentinel2_', '').replace('.tif', '')
    sentinel_path = os.path.join(sentinel_dir, filename)
    dem_path = os.path.join(dem_dir, f'DEM_{gunung_name}.tif')
    thermal_path = os.path.join(thermal_dir, f'Thermal_RGB_{gunung_name}.tif')

    if not os.path.exists(dem_path) or not os.path.exists(thermal_path):
        print(f"[SKIP] Data tidak lengkap untuk {gunung_name}")
        continue

    # ---------- Baca Sentinel (NDVI) ----------
    with rasterio.open(sentinel_path) as src:
        B4 = src.read(4).astype('float32')  # Red
        B8 = src.read(8).astype('float32')  # NIR
        transform = src.transform
        crs = src.crs

    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (B8 - B4) / (B8 + B4)
        ndvi[(B8 + B4) == 0] = np.nan
    ndvi = np.clip(ndvi, -1, 1)

    # ---------- Baca DEM ----------
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1).astype('float32')

        if dem_data.shape != ndvi.shape or dem_src.transform != transform or dem_src.crs != crs:
            from rasterio.warp import reproject, Resampling
            dem_resampled = np.full_like(ndvi, np.nan, dtype='float32')
            reproject(
                source=dem_data,
                destination=dem_resampled,
                src_transform=dem_src.transform,
                src_crs=dem_src.crs,
                dst_transform=transform,
                dst_crs=crs,
                resampling=Resampling.bilinear
            )
        else:
            dem_resampled = dem_data

    # ---------- Baca Thermal ----------
    with rasterio.open(thermal_path) as th_src:
        thermal_data = th_src.read(1).astype('float32')

        if thermal_data.shape != ndvi.shape or th_src.transform != transform or th_src.crs != crs:
            from rasterio.warp import reproject, Resampling
            thermal_resampled = np.full_like(ndvi, np.nan, dtype='float32')
            reproject(
                source=thermal_data,
                destination=thermal_resampled,
                src_transform=th_src.transform,
                src_crs=th_src.crs,
                dst_transform=transform,
                dst_crs=crs,
                resampling=Resampling.bilinear
            )
        else:
            thermal_resampled = thermal_data

    # ==========================
    # 1) K-MEANS NDVI (2 cluster)
    # ==========================
    flat_ndvi = ndvi.reshape(-1, 1)
    valid_ndvi_mask = ~np.isnan(flat_ndvi[:, 0])
    valid_ndvi_vals = flat_ndvi[valid_ndvi_mask]

    if valid_ndvi_vals.size == 0:
        print(f"[SKIP] Semua NDVI NaN: {gunung_name}")
        continue

    kmeans_ndvi = KMeans(n_clusters=2, random_state=42, n_init=10)
    ndvi_labels_flat = np.zeros_like(flat_ndvi[:, 0], dtype=np.uint8)
    ndvi_labels_flat[valid_ndvi_mask] = kmeans_ndvi.fit_predict(valid_ndvi_vals)

    ndvi_means = []
    for i in range(2):
        vals = valid_ndvi_vals[ndvi_labels_flat[valid_ndvi_mask] == i]
        ndvi_means.append(vals.mean() if vals.size > 0 else np.inf)
    low_ndvi_cluster = int(np.argmin(ndvi_means))

    ndvi_low_mask = (ndvi_labels_flat.reshape(ndvi.shape) == low_ndvi_cluster)

    # ==========================
    # 2) K-MEANS Thermal
    # ==========================
    flat_th = thermal_resampled.reshape(-1, 1)
    valid_th_mask = ~np.isnan(flat_th[:, 0])
    valid_th_vals = flat_th[valid_th_mask]

    if valid_th_vals.size == 0:
        print(f"[SKIP] Semua Thermal NaN: {gunung_name}")
        continue

    kmeans_th = KMeans(n_clusters=2, random_state=42, n_init=10)
    th_labels_flat = np.zeros_like(flat_th[:, 0], dtype=np.uint8)
    th_labels_flat[valid_th_mask] = kmeans_th.fit_predict(valid_th_vals)

    th_means = []
    for i in range(2):
        vals = valid_th_vals[th_labels_flat[valid_th_mask] == i]
        th_means.append(vals.mean() if vals.size > 0 else -np.inf)
    high_th_cluster = int(np.argmax(th_means))

    th_high_mask = (th_labels_flat.reshape(ndvi.shape) == high_th_cluster)

    # ==========================
    # 3) K-MEANS DEM dalam area NDVI rendah & Thermal tinggi
    # ==========================
    combined_mask = ndvi_low_mask & th_high_mask
    dem_on_combined = np.where(combined_mask, dem_resampled, np.nan)

    flat_dem = dem_on_combined.reshape(-1, 1)
    valid_dem_mask = ~np.isnan(flat_dem[:, 0])
    valid_dem_vals = flat_dem[valid_dem_mask]

    if valid_dem_vals.size == 0:
        print(f"[SKIP] Tidak ada piksel valid untuk DEM di area NDVI rendah & Thermal tinggi: {gunung_name}")
        continue

    kmeans_dem = KMeans(n_clusters=2, random_state=42, n_init=10)
    dem_labels_flat = np.zeros_like(flat_dem[:, 0], dtype=np.uint8)
    dem_labels_flat[valid_dem_mask] = kmeans_dem.fit_predict(valid_dem_vals)

    dem_means = []
    for i in range(2):
        vals = valid_dem_vals[dem_labels_flat[valid_dem_mask] == i]
        dem_means.append(vals.mean() if vals.size > 0 else -np.inf)
    high_dem_cluster = int(np.argmax(dem_means))

    dem_high_mask = (dem_labels_flat.reshape(ndvi.shape) == high_dem_cluster)
    final_mask = combined_mask & dem_high_mask

    if not np.any(final_mask):
        print(f"[WARN] Final mask kosong: {gunung_name}. Pusat tidak dapat ditentukan.")
        continue

    # ==========================
    # 4) CENTER OF MASS
    # ==========================
    cy, cx = center_of_mass(final_mask.astype(np.uint8))
    if np.isnan(cy) or np.isnan(cx):
        print(f"[WARN] Center of mass tidak valid: {gunung_name}")
        continue
    cy_i, cx_i = int(round(cy)), int(round(cx))
    lon, lat = rasterio.transform.xy(transform, cy_i, cx_i)

    hasil_deteksi_metode6[gunung_name] = {'detected_lat': float(lat), 'detected_lon': float(lon)}

    # ==========================
    # 5) VISUALISASI
    # ==========================
    true_color = create_true_color_image(sentinel_path)
    fig, axs = plt.subplots(2, 4, figsize=(20, 10), constrained_layout=True)

    # Baris 1
    axs[0, 0].imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    axs[0, 0].set_title('NDVI Map'); axs[0, 0].axis('off')

    axs[0, 1].imshow(ndvi_low_mask, cmap='gray')
    axs[0, 1].set_title('Mask NDVI Rendah'); axs[0, 1].axis('off')

    axs[0, 2].imshow(thermal_resampled, cmap='inferno')
    axs[0, 2].set_title('Thermal Map'); axs[0, 2].axis('off')

    axs[0, 3].imshow(th_high_mask, cmap='gray')
    axs[0, 3].set_title('Mask Thermal Tinggi'); axs[0, 3].axis('off')

    # Baris 2
    axs[1, 0].imshow(combined_mask, cmap='gray')
    axs[1, 0].set_title('Mask Gabungan (NDVI + Thermal)'); axs[1, 0].axis('off')

    axs[1, 1].imshow(dem_resampled, cmap='terrain')
    axs[1, 1].set_title('DEM Map'); axs[1, 1].axis('off')

    axs[1, 2].imshow(final_mask, cmap='gray')
    axs[1, 2].set_title('Final Mask (NDVI + Thermal + DEM)'); axs[1, 2].axis('off')

    axs[1, 3].imshow(true_color)
    axs[1, 3].scatter(cx, cy, c='red', s=50, label='Titik Pusat')
    axs[1, 3].legend()
    axs[1, 3].set_title('True Color + Titik Pusat'); axs[1, 3].axis('off')

    fig.suptitle(f'{gunung_name} | Metode 6\nKoordinat: {lat:.5f}, {lon:.5f}', fontsize=16)
    plt.savefig(os.path.join(output_dir, f'{gunung_name}.png'), dpi=150, bbox_inches='tight')
    plt.close()


# -----------------------------
# Simpan CSV
# -----------------------------
csv_path = os.path.join(output_dir, 'hasil_deteksi_metode6.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['nama_gunung', 'lat', 'lon'])
    for name, coord in hasil_deteksi_metode6.items():
        writer.writerow([name, coord['detected_lat'], coord['detected_lon']])


c:\Users\Naomi\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


# Evaluation

## Import KRB Center Point Data

In [16]:
import pandas as pd

# Membaca CSV
csv_path = 'List KRB center point.csv'
df_krb = pd.read_csv(csv_path)

# Membuat dictionary dari data CSV
krb_center_points = {}

for idx, row in df_krb.iterrows():
    nama = row['Nama']
    lat_lon_str = row['Titik Koordinat Pusat']
    lat_str, lon_str = lat_lon_str.split(',')
    lat, lon = float(lat_str.strip()), float(lon_str.strip())

    krb_center_points[nama] = {
        'detected_lat': lat,
        'detected_lon': lon
    }

# Contoh output
print("Dictionary Koordinat KRB:")
for nama, koordinat in krb_center_points.items():
    print(f"{nama}: {koordinat['detected_lat']}, {koordinat['detected_lon']}")


Dictionary Koordinat KRB:
Agung: -8.341953075466813, 115.50750732421875
Ambang: 0.7535717180105911, 124.42119598388675
Anak Krakatau: -6.1016391293547425, 105.4241180419922
Anak Ranakah: -8.633806608897267, 120.53272247314456
Arjuno Welirang: -7.733445463620818, 112.57484436035155
Awu: 3.682345501935164, 125.4521942138672
Banda Api: -4.52269310446871, 129.88105773925784
Batur: -8.24563904165236, 115.36931991577148
Batutara: -7.789914903590686, 123.58777999877933
Bromo: -7.941596147894652, 112.95181274414064
Bur Ni Telong: 4.768730969283216, 96.82113647460939
Ciremai: -6.895752309876321, 108.4079360961914
Colo: -0.16891455023785007, 121.60766601562503
Dempo: -4.01564458761465, 103.11973571777345
Dieng: -7.199681955820926, 109.84199523925783
Dukono: 1.700571185447282, 127.87742614746094
Ebulobo: -8.815698116551511, 121.19104385375978
Egon: -8.677421123289987, 122.45412826538086
Galunggung: -7.256561216674466, 108.07594299316408
Gamalama: 0.8084982528736103, 127.32982635498048
Gamkonora: 

## Final Visualization (True Color with 3 Points from Each Methods + 1 KRB Center Point)

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Folder input dan output
input_dir = '../../DATA/SENTINEL v6/'
output_dir_viz = 'final'
os.makedirs(output_dir_viz, exist_ok=True)

# Offset kecil untuk menghindari titik bertumpuk
offsets = {
    'krb': (0, 0),
    'metode1': (2, 2),
    'metode2': (-2, 2),
    'metode3': (2, -2)
}

# Loop per file citra
for nama_krb in krb_center_points.keys():
    # Modify this line to handle both hyphen '-' and space ' '
    nama_lookup = nama_krb.replace('Laki-laki', 'Laki_Laki').replace(' ', '_').replace('-', '_')


    if nama_lookup in hasil_deteksi_metode1 and nama_lookup in hasil_deteksi_metode2 and nama_lookup in hasil_deteksi_metode3:
        filepath = os.path.join(input_dir, f'Sentinel2_{nama_lookup}.tif')

        if os.path.exists(filepath):
            with rasterio.open(filepath) as src:
                band4 = src.read(4).astype('float32')  # Red
                band3 = src.read(3).astype('float32')  # Green
                band2 = src.read(2).astype('float32')  # Blue

                # Buat true color
                true_color = np.dstack((band4, band3, band2))
                true_color = np.nan_to_num(true_color, nan=0)
                if true_color.max() > 0:
                    true_color = true_color / true_color.max()

                # Konversi koordinat ke piksel
                def geo_to_pixel(lon, lat):
                    row, col = src.index(lon, lat)
                    return col, row  # Note: col = x, row = y di gambar

                # Ambil titik
                lat_krb, lon_krb = krb_center_points[nama_krb]['detected_lat'], krb_center_points[nama_krb]['detected_lon']
                lat_m1, lon_m1 = hasil_deteksi_metode1[nama_lookup]['detected_lat'], hasil_deteksi_metode1[nama_lookup]['detected_lon']
                lat_m2, lon_m2 = hasil_deteksi_metode2[nama_lookup]['detected_lat'], hasil_deteksi_metode2[nama_lookup]['detected_lon']
                lat_m3, lon_m3 = hasil_deteksi_metode3[nama_lookup]['detected_lat'], hasil_deteksi_metode3[nama_lookup]['detected_lon']

                x_krb, y_krb = geo_to_pixel(lon_krb, lat_krb)
                x_m1, y_m1 = geo_to_pixel(lon_m1, lat_m1)
                x_m2, y_m2 = geo_to_pixel(lon_m2, lat_m2)
                x_m3, y_m3 = geo_to_pixel(lon_m3, lat_m3)

                # Plot
                plt.figure(figsize=(10, 10))
                plt.imshow(true_color)

                plt.scatter(x_krb + offsets['krb'][0], y_krb + offsets['krb'][1], c='cyan', s=50, label='Titik KRB')
                plt.scatter(x_m1 + offsets['metode1'][0], y_m1 + offsets['metode1'][1], c='red', s=50, label='Metode 1')
                plt.scatter(x_m2 + offsets['metode2'][0], y_m2 + offsets['metode2'][1], c='pink', s=50, label='Metode 2')
                plt.scatter(x_m3 + offsets['metode3'][0], y_m3 + offsets['metode3'][1], c='yellow', s=50, label='Metode 3')

                plt.legend(loc='lower right')
                plt.title(f'Visualisasi Hasil Deteksi Pusat Gunung {nama_krb}', fontsize=16, fontweight='bold')
                plt.axis('off')

                # Save
                output_path = os.path.join(output_dir_viz, f'{nama_krb}.png')
                plt.savefig(output_path, bbox_inches='tight')
                plt.close()

print("Semua visualisasi titik telah selesai disimpan!")


Semua visualisasi titik telah selesai disimpan!


## Final Visualization (True Color with 5 Points from Each Methods + 1 KRB Center Point)

In [19]:
import os
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Folder input dan output
input_dir = '../DATA/SENTINEL 2/'
output_dir_viz = 'final_5metode'
os.makedirs(output_dir_viz, exist_ok=True)

# Offset kecil untuk menghindari titik bertumpuk
offsets = {
    'krb': (0, 0),
    'metode1': (2, 2),
    'metode2': (-2, 2),
    'metode3': (2, -2),
    'metode4': (-2, -2),
    'metode5': (0, 4)
}

# Loop per file citra
for nama_krb in krb_center_points.keys():
    # Modify nama untuk lookup
    nama_lookup = nama_krb.replace('Laki-laki', 'Laki_Laki').replace(' ', '_').replace('-', '_')

    # Pastikan semua metode ada
    if (nama_lookup in hasil_deteksi_metode1 and
        nama_lookup in hasil_deteksi_metode2 and
        nama_lookup in hasil_deteksi_metode3 and
        nama_lookup in hasil_deteksi_metode4 and
        nama_lookup in hasil_deteksi_metode5):

        filepath = os.path.join(input_dir, f'Sentinel2_{nama_lookup}.tif')

        if os.path.exists(filepath):
            with rasterio.open(filepath) as src:
                band4 = src.read(4).astype('float32')  # Red
                band3 = src.read(3).astype('float32')  # Green
                band2 = src.read(2).astype('float32')  # Blue

                # True color
                true_color = np.dstack((band4, band3, band2))
                true_color = np.nan_to_num(true_color, nan=0)
                if true_color.max() > 0:
                    true_color = true_color / true_color.max()

                # Fungsi geo -> pixel
                def geo_to_pixel(lon, lat):
                    row, col = src.index(lon, lat)
                    return col, row

                # Ambil titik koordinat semua metode
                lat_krb, lon_krb = krb_center_points[nama_krb]['detected_lat'], krb_center_points[nama_krb]['detected_lon']
                lat_m1, lon_m1 = hasil_deteksi_metode1[nama_lookup]['detected_lat'], hasil_deteksi_metode1[nama_lookup]['detected_lon']
                lat_m2, lon_m2 = hasil_deteksi_metode2[nama_lookup]['detected_lat'], hasil_deteksi_metode2[nama_lookup]['detected_lon']
                lat_m3, lon_m3 = hasil_deteksi_metode3[nama_lookup]['detected_lat'], hasil_deteksi_metode3[nama_lookup]['detected_lon']
                lat_m4, lon_m4 = hasil_deteksi_metode4[nama_lookup]['detected_lat'], hasil_deteksi_metode4[nama_lookup]['detected_lon']
                lat_m5, lon_m5 = hasil_deteksi_metode5[nama_lookup]['detected_lat'], hasil_deteksi_metode5[nama_lookup]['detected_lon']

                # Konversi ke pixel
                x_krb, y_krb = geo_to_pixel(lon_krb, lat_krb)
                x_m1, y_m1 = geo_to_pixel(lon_m1, lat_m1)
                x_m2, y_m2 = geo_to_pixel(lon_m2, lat_m2)
                x_m3, y_m3 = geo_to_pixel(lon_m3, lat_m3)
                x_m4, y_m4 = geo_to_pixel(lon_m4, lat_m4)
                x_m5, y_m5 = geo_to_pixel(lon_m5, lat_m5)

                # Plot
                plt.figure(figsize=(10, 10))
                plt.imshow(true_color)

                plt.scatter(x_krb + offsets['krb'][0], y_krb + offsets['krb'][1], c='cyan', s=50, label='Titik KRB')
                plt.scatter(x_m1 + offsets['metode1'][0], y_m1 + offsets['metode1'][1], c='red', s=50, label='Metode 1')
                plt.scatter(x_m2 + offsets['metode2'][0], y_m2 + offsets['metode2'][1], c='pink', s=50, label='Metode 2')
                plt.scatter(x_m3 + offsets['metode3'][0], y_m3 + offsets['metode3'][1], c='yellow', s=50, label='Metode 3')
                plt.scatter(x_m4 + offsets['metode4'][0], y_m4 + offsets['metode4'][1], c='green', s=50, label='Metode 4')
                plt.scatter(x_m5 + offsets['metode5'][0], y_m5 + offsets['metode5'][1], c='orange', s=50, label='Metode 5')

                plt.legend(loc='lower right')
                plt.title(f'Visualisasi Hasil Deteksi Pusat Gunung {nama_krb}', fontsize=16, fontweight='bold')
                plt.axis('off')

                # Save
                output_path = os.path.join(output_dir_viz, f'{nama_krb}.png')
                plt.savefig(output_path, bbox_inches='tight')
                plt.close()

print("Semua visualisasi 5 metode telah selesai disimpan!")


Semua visualisasi 5 metode telah selesai disimpan!


## Calculate Distance for All Methods

In [9]:
import numpy as np
import pandas as pd

# Fungsi untuk menghitung jarak Haversine (kilometer)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius bumi dalam kilometer
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

# List hasil semua
hasil_perbandingan = []

for nama_krb in krb_center_points.keys():
    nama_lookup = nama_krb.replace('Laki-laki','Laki_Laki').replace(' ', '_')  # hanya untuk lookup ke hasil_deteksi

    if nama_lookup in hasil_deteksi_metode1 and nama_lookup in hasil_deteksi_metode2 and nama_lookup in hasil_deteksi_metode3:
        lat_krb = krb_center_points[nama_krb]['detected_lat']
        lon_krb = krb_center_points[nama_krb]['detected_lon']

        jarak1 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode1[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode1[nama_lookup]['detected_lon'])
        jarak2 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode2[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode2[nama_lookup]['detected_lon'])
        jarak3 = haversine(lat_krb, lon_krb,
                           hasil_deteksi_metode3[nama_lookup]['detected_lat'],
                           hasil_deteksi_metode3[nama_lookup]['detected_lon'])

        jarak1 = round(jarak1, 2)
        jarak2 = round(jarak2, 2)
        jarak3 = round(jarak3, 2)

        hasil_perbandingan.append({
            'Gunung': nama_krb,   # pakai nama asli dari KRB (tetap spasi)
            'M1': jarak1,
            'M2': jarak2,
            'M3': jarak3
        })

# Buat DataFrame
df_perbandingan = pd.DataFrame(hasil_perbandingan)

# Fungsi pilih metode terbaik dan terburuk
def pilih_metode(row):
    values = {
        'Metode 1': row['M1'],
        'Metode 2': row['M2'],
        'Metode 3': row['M3']
    }
    metode_terbaik = min(values, key=values.get)
    metode_terburuk = max(values, key=values.get)
    jarak_min = round(min(values.values()), 2)
    jarak_max = round(max(values.values()), 2)
    return pd.Series([jarak_min, metode_terbaik, jarak_max, metode_terburuk],
                     index=['Min', 'Best Method', 'Max', 'Worst Method'])

# Apply ke DataFrame
df_perbandingan[['Min', 'Best Method', 'Max', 'Worst Method']] = df_perbandingan.apply(pilih_metode, axis=1)

# Print hasil
print(df_perbandingan)

# (Opsional) Simpan ke CSV
df_perbandingan.to_csv('(NEW)hasil_perbandingan_fix.csv', index=False)


              Gunung    M1    M2    M3   Min Best Method   Max Worst Method
0              Agung  0.46  0.31  0.31  0.31    Metode 2  0.46     Metode 1
1             Ambang  0.64  0.45  0.79  0.45    Metode 2  0.79     Metode 3
2      Anak Krakatau  0.17  0.40  0.27  0.17    Metode 1  0.40     Metode 2
3       Anak Ranakah  1.10  0.29  0.45  0.29    Metode 2  1.10     Metode 1
4    Arjuno Welirang  2.39  3.48  2.39  2.39    Metode 1  3.48     Metode 2
..               ...   ...   ...   ...   ...         ...   ...          ...
63           Tambora  2.47  2.64  2.91  2.47    Metode 1  2.91     Metode 3
64          Tandikat  4.23  5.02  4.39  4.23    Metode 1  5.02     Metode 2
65          Tangkoko  3.51  4.31  3.14  3.14    Metode 3  4.31     Metode 2
66  Tangkuban Parahu  5.80  8.80  2.07  2.07    Metode 3  8.80     Metode 2
67           Wurlali  0.12  0.10  0.31  0.10    Metode 2  0.31     Metode 3

[68 rows x 8 columns]


## Mengecek apakah ada titik KRB yang tidak termasuk dalam citra

In [ ]:
import rasterio
import os

# Folder input
input_dir = '../../DATA/NEW AREA/'

# Untuk menyimpan hasil
krb_outside_image = []

for nama_krb in krb_center_points.keys():
    nama_lookup = nama_krb.replace(' ', '_').replace('-', '_')

    filepath = os.path.join(input_dir, f'Sentinel2_{nama_lookup}.tif')

    if os.path.exists(filepath):
        with rasterio.open(filepath) as src:
            bounds = src.bounds  # Bounding box citra: left, bottom, right, top

        lat_krb = krb_center_points[nama_krb]['detected_lat']
        lon_krb = krb_center_points[nama_krb]['detected_lon']

        # Cek apakah lon dan lat KRB masuk dalam bounding box
        if not (bounds.left <= lon_krb <= bounds.right and bounds.bottom <= lat_krb <= bounds.top):
            krb_outside_image.append(nama_krb)

# Tampilkan hasil
if krb_outside_image:
    print("Titik KRB berikut berada di luar area citra:")
    for nama in krb_outside_image:
        print("-", nama)
else:
    print("Semua titik KRB berada dalam area citra.")


Titik KRB berikut berada di luar area citra:
- Dieng
- Tandikat
- Tangkoko


## New Method Comparisson (Method 1-5)

In [2]:
import pandas as pd
import numpy as np

# Fungsi Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    dlat, dlon = lat2_rad - lat1_rad, lon2_rad - lon1_rad
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

# Baca CSV KRB
df_krb = pd.read_csv('List KRB center point.csv')
krb_points = {}
for idx, row in df_krb.iterrows():
    nama = row['Nama']
    lat_str, lon_str = row['Titik Koordinat Pusat'].split(',')
    krb_points[nama] = (float(lat_str.strip()), float(lon_str.strip()))

# Fungsi baca hasil metode dan ubah jadi dictionary
def read_hasil_csv(path):
    df = pd.read_csv(path)
    hasil = {}
    for idx, row in df.iterrows():
        nama_lookup = row['nama_gunung'].replace(' ', '_').replace('-', '_')
        hasil[nama_lookup] = (row['lat'], row['lon'])
    return hasil

# Baca hasil metode 4 & 5
hasil_metode4 = read_hasil_csv('Metode 4/hasil_deteksi_metode4.csv')
hasil_metode5 = read_hasil_csv('Metode 5/hasil_deteksi_metode5.csv')

# Baca hasil metode 1–3 dari CSV sebelumnya (opsional, kalau sudah ada)
df_m123 = pd.read_csv('Previous Result.csv')

# Siapkan dictionary metode 1–3
def df_to_dict_m123(df):
    d = {}
    for idx, row in df.iterrows():
        nama_lookup = row['Gunung'].replace(' ', '_').replace('-', '_')
        d[nama_lookup] = (row['M1'], row['M2'], row['M3'])
    return d

hasil_metode123 = df_to_dict_m123(df_m123)

# Hitung jarak semua metode
hasil_perbandingan = []

for nama_krb, (lat_krb, lon_krb) in krb_points.items():
    nama_lookup = nama_krb.replace(' ', '_').replace('-', '_')
    
    if nama_lookup in hasil_metode123 and nama_lookup in hasil_metode4 and nama_lookup in hasil_metode5:
        # Metode 1-3 sudah berupa jarak
        m1, m2, m3 = hasil_metode123[nama_lookup]
        
        # Hitung jarak metode 4 & 5
        lat4, lon4 = hasil_metode4[nama_lookup]
        lat5, lon5 = hasil_metode5[nama_lookup]
        m4 = round(haversine(lat_krb, lon_krb, lat4, lon4), 2)
        m5 = round(haversine(lat_krb, lon_krb, lat5, lon5), 2)
        
        # Simpan hasil
        data = {
            'Gunung': nama_krb,
            'M1': m1,
            'M2': m2,
            'M3': m3,
            'M4': m4,
            'M5': m5
        }
        # Min, max, best, worst, avg
        jarak_list = [m1, m2, m3, m4, m5]
        data['Min'] = round(min(jarak_list),2)
        data['Best Method'] = f"Metode {jarak_list.index(min(jarak_list))+1}"
        data['Max'] = round(max(jarak_list),2)
        data['Worst Method'] = f"Metode {jarak_list.index(max(jarak_list))+1}"
        data['Average'] = round(sum(jarak_list)/len(jarak_list),2)
        
        hasil_perbandingan.append(data)

# Buat DataFrame
df_final = pd.DataFrame(hasil_perbandingan)
print(df_final)

# Simpan ke CSV
df_final.to_csv('hasil_perbandingan_metode1_5.csv', index=False)


              Gunung    M1    M2    M3    M4    M5   Min Best Method   Max  \
0              Agung  0.46  0.31  0.31  0.42  0.18  0.18    Metode 5  0.46   
1             Ambang  0.64  0.45  0.79  0.25  0.35  0.25    Metode 4  0.79   
2      Anak Krakatau  0.17  0.40  0.27  0.20  0.73  0.17    Metode 1  0.73   
3       Anak Ranakah  1.10  0.29  0.45  0.46  0.15  0.15    Metode 5  1.10   
4    Arjuno Welirang  2.39  3.48  2.39  2.47  2.21  2.21    Metode 5  3.48   
..               ...   ...   ...   ...   ...   ...   ...         ...   ...   
62           Tambora  2.47  2.64  2.91  0.33  0.22  0.22    Metode 5  2.91   
63          Tandikat  1.38  2.28  1.91  0.74  0.99  0.74    Metode 4  2.28   
64          Tangkoko  0.32  0.43  0.09  0.56  0.09  0.09    Metode 3  0.56   
65  Tangkuban Parahu  5.80  8.80  2.07  4.17  5.30  2.07    Metode 3  8.80   
66           Wurlali  0.12  0.10  0.31  0.22  0.12  0.10    Metode 2  0.31   

   Worst Method  Average  
0      Metode 1     0.34  
1      Me

## New Method Comparrison (Method 1-6)

In [2]:
import pandas as pd
import numpy as np

# Fungsi Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    dlat, dlon = lat2_rad - lat1_rad, lon2_rad - lon1_rad
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

# Baca CSV KRB
df_krb = pd.read_csv('List KRB center point.csv')
krb_points = {}
for idx, row in df_krb.iterrows():
    nama = row['Nama']
    lat_str, lon_str = row['Titik Koordinat Pusat'].split(',')
    krb_points[nama] = (float(lat_str.strip()), float(lon_str.strip()))

# Fungsi baca hasil metode dan ubah jadi dictionary
def read_hasil_csv(path):
    df = pd.read_csv(path)
    hasil = {}
    for idx, row in df.iterrows():
        nama_lookup = row['nama_gunung'].replace(' ', '_').replace('-', '_')
        hasil[nama_lookup] = (row['lat'], row['lon'])
    return hasil

# Baca hasil metode 4 & 5 & 6
hasil_metode4 = read_hasil_csv('Metode 4/hasil_deteksi_metode4.csv')
hasil_metode5 = read_hasil_csv('Metode 5/hasil_deteksi_metode5.csv')
hasil_metode6 = read_hasil_csv('Metode 6/hasil_deteksi_metode6.csv')

# Baca hasil metode 1–3 dari CSV sebelumnya (opsional, kalau sudah ada)
df_m123 = pd.read_csv('Previous Result.csv')

# Siapkan dictionary metode 1–3
def df_to_dict_m123(df):
    d = {}
    for idx, row in df.iterrows():
        nama_lookup = row['Gunung'].replace(' ', '_').replace('-', '_')
        d[nama_lookup] = (row['M1'], row['M2'], row['M3'])
    return d

hasil_metode123 = df_to_dict_m123(df_m123)

# Hitung jarak semua metode
hasil_perbandingan = []

for nama_krb, (lat_krb, lon_krb) in krb_points.items():
    nama_lookup = nama_krb.replace(' ', '_').replace('-', '_')
    
    if nama_lookup in hasil_metode123 and nama_lookup in hasil_metode4 and nama_lookup in hasil_metode5 and nama_lookup in hasil_metode6:
        # Metode 1-3 sudah berupa jarak
        m1, m2, m3 = hasil_metode123[nama_lookup]
        
        # Hitung jarak metode 4 & 5
        lat4, lon4 = hasil_metode4[nama_lookup]
        lat5, lon5 = hasil_metode5[nama_lookup]
        lat6, lon6 = hasil_metode6[nama_lookup]
        m4 = round(haversine(lat_krb, lon_krb, lat4, lon4), 2)
        m5 = round(haversine(lat_krb, lon_krb, lat5, lon5), 2)
        m6 = round(haversine(lat_krb, lon_krb, lat6, lon6), 2)
        
        # Simpan hasil
        data = {
            'Gunung': nama_krb,
            'M1': m1,
            'M2': m2,
            'M3': m3,
            'M4': m4,
            'M5': m5,
            'M6': m6
        }
        # Min, max, best, worst, avg
        jarak_list = [m1, m2, m3, m4, m5, m6]
        data['Min'] = round(min(jarak_list),2)
        data['Best Method'] = f"Metode {jarak_list.index(min(jarak_list))+1}"
        data['Max'] = round(max(jarak_list),2)
        data['Worst Method'] = f"Metode {jarak_list.index(max(jarak_list))+1}"
        data['Average'] = round(sum(jarak_list)/len(jarak_list),2)
        
        hasil_perbandingan.append(data)

# Buat DataFrame
df_final = pd.DataFrame(hasil_perbandingan)
print(df_final)

# Simpan ke CSV
df_final.to_csv('hasil_perbandingan_metode1_6.csv', index=False)


              Gunung    M1    M2    M3    M4    M5    M6   Min Best Method  \
0              Agung  0.46  0.31  0.31  0.42  0.18  0.29  0.18    Metode 5   
1             Ambang  0.64  0.45  0.79  0.25  0.35  0.33  0.25    Metode 4   
2      Anak Krakatau  0.17  0.40  0.27  0.20  0.73  0.26  0.17    Metode 1   
3       Anak Ranakah  1.10  0.29  0.45  0.46  0.15  0.29  0.15    Metode 5   
4    Arjuno Welirang  2.39  3.48  2.39  2.47  2.21  2.07  2.07    Metode 6   
..               ...   ...   ...   ...   ...   ...   ...   ...         ...   
62           Tambora  2.47  2.64  2.91  0.33  0.22  1.09  0.22    Metode 5   
63          Tandikat  1.38  2.28  1.91  0.74  0.99  2.02  0.74    Metode 4   
64          Tangkoko  0.32  0.43  0.09  0.56  0.09  0.08  0.08    Metode 6   
65  Tangkuban Parahu  5.80  8.80  2.07  4.17  5.30  6.34  2.07    Metode 3   
66           Wurlali  0.12  0.10  0.31  0.22  0.12  0.19  0.10    Metode 2   

     Max Worst Method  Average  
0   0.46     Metode 1     0.33